Personality Classifications

SIMPING

Openness to experience (inventive/curious vs. consistent/cautious)

Conscientiousness (efficient/organized vs. easy-going/careless)

Extraversion (outgoing/energetic vs. solitary/reserved)

Agreeableness (friendly/compassionate vs. challenging/detached)

Neuroticism (sensitive/nervous vs. secure/confident)

Characteristics of Songs We Will Be Looking At

Tempo

Danceability

Energy 

Instrumentalness



In [0]:
%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals

from google.colab import files
import tensorflow as tf
import pandas as pd
import io

In [0]:
col_names = ['Danceability', 'Tempo', 'Energy', 'Instrumentalness', 'Label']
personality_types = ['O','C', 'E', 'A', 'N']


In [28]:
# train_path = tf.keras.utils.get_file(
#     "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
# test_path = tf.keras.utils.get_file(
#     "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
train_path = files.upload()
test_path  = files.upload()

train = pd.read_csv(io.StringIO(train_path['tracks.csv'].decode('utf-8')))
test = pd.read_csv(io.StringIO(test_path['test.csv'].decode('utf-8')))

Saving tracks.csv to tracks (6).csv


Saving test.csv to test (5).csv


In [29]:
training_label = train.pop('Label')
test_label = test.pop('Label')
train.head()

,Danceability,Tempo,Instrumentalness,Energy
0,0.521,129.900,0.001240,0.453
1,0.515,88.964,0.005980,0.479
2,0.315,88.045,0.000211,0.220
3,0.732,102.087,0.000000,0.243
4,0.498,118.917,0.000000,0.235


In [0]:
#define input function for model
def input_fn(features, labels, training=True, batch_size=256):
    #Create dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)


In [31]:
#Feature columns
feat_cols = []
for k in train.keys():
    feat_cols.append(tf.feature_column.numeric_column(key=k))
print(feat_cols)


[NumericColumn(key='Danceability', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Tempo', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Instrumentalness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Energy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


At this point we can choose from 2 different classifiers. A DNN(Deep Neural Network) 


In [32]:
#DNN below
classifier = tf.estimator.DNNClassifier(
    hidden_units=[30,10],
    feature_columns=feat_cols,
    n_classes=5,
)
#n_classes must be equal to 5 for personality classifier


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp49arqb7o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [33]:
classifier.train(input_fn=lambda: input_fn(train,training_label, training=True), steps= 5000)


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp49arqb7o/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 11.50783, step = 0
INFO:

In [34]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test,test_label, training=False))

print('\n Test accuracy : {accuracy:0.3f}\n'.format(**eval_result))
#Test accuract : 0.967 for testing data

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-27T23:32:05Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp49arqb7o/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.27416s
INFO:tensorflow:Finished evaluation at 2020-03-27-23:32:05
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.0, average_loss 

ERROR! Session/line number was not unique in database. History logging moved to new session 59


NameError: ignored